In [1]:
import time 

from IPython.core.display import display
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
plt.style.use('ggplot')
import numpy as np
import scipy
import seaborn as sns
sns.set(font='Osaka')
import pandas as pd

In [2]:
t0 = time.time()
data_dir = '../data/'
file = 'df_item_transaction.csv'
df_item_transaction = pd.read_csv(data_dir + file)
print('Load in ', time.time() - t0)

Load in  1.008876085281372


In [4]:
country = 'United Kingdom'
df_uk = \
df_item_transaction[df_item_transaction['Country'] == country].drop('Unnamed: 0', axis=1)

# CustomerID to Nominal
df_uk['CustomerID'] = df_uk['CustomerID'].fillna('NaN')

# InvoiceDate to Datetime
df_uk['InvoiceDate'] = pd.to_datetime(df_uk['InvoiceDate'])

assert df_uk['StockCode'].nunique() == 3910
assert df_uk['CustomerID'].nunique() == 3918

print('# of distinct items =', df_uk['StockCode'].nunique())
print('# of distinct customers =', df_uk['CustomerID'].nunique())

# of distinct items = 3910
# of distinct customers = 3918


In [5]:
# Customer ごとにデータを集計　
# Recency
# Frequency = #ofTransactions
# Monetary = sum(Sales)

# Sales: min, max, median, mean, sum
# Quantity: min, max, median, mean, sum
# Quantity / Transaction: min, max, median, mean
# Unitprice: min, max, median, mean

# Item / Transaction = 商品の種類 / Transaction: min, max, median, mean


# 各商品をいくつ購入したか？　(3,910 items hence the same # of columns)

# TODO:
# ある商品を購入する顧客は Frequency が大きいといった関係が発見できるか？
# RMF 分析で優良顧客を見つけ出し、彼らの 2月, 4月の


# Monetary
df_uk_per_customer = \
pd.DataFrame(df_uk.groupby('CustomerID')['Sales'].sum()).reset_index()\
.rename(columns={'Sales': 'Monetary'})

# Frequency
df_uk_per_customer['Frequency'] = \
pd.DataFrame(df_uk.groupby('CustomerID')['InvoiceNo'].nunique()).reset_index()['InvoiceNo']
# small test
test_customer = 12748
assert df_uk_per_customer[df_uk_per_customer['CustomerID'] == test_customer]['Frequency'].values\
            == df_uk[df_uk['CustomerID'] == test_customer]['InvoiceNo'].nunique()

# Recency
# 基準日
df_uk_per_customer['ReferenceDate'] = \
    pd.to_datetime(df_item_transaction['InvoiceDate']).max()
# 最終購入日
df_uk_per_customer['LastTransactionDate'] = \
    pd.DataFrame(df_uk.groupby('CustomerID')['InvoiceDate'].max()).reset_index()['InvoiceDate']
# 基準日 - 最終購入日
df_uk_per_customer['Recency(Day)'] = \
(df_uk_per_customer['ReferenceDate'].dt.date - df_uk_per_customer['LastTransactionDate'].dt.date).dt.days


# sum(Quantity)
df_uk_per_customer['QuantitySum'] = pd.DataFrame(df_uk.groupby('CustomerID')['Quantity'].sum()).reset_index()['Quantity']

# Quantity / Transaction
df_uk_per_customer['QuantityPerTransaction'] = df_uk_per_customer['QuantitySum'] / df_uk_per_customer['Frequency']

# Sales / Transaction
df_uk_per_customer['Sales/Transaction'] = df_uk_per_customer['Monetary'] / df_uk_per_customer['Frequency']

In [6]:
display(df_uk_per_customer.head(3))

,CustomerID,Monetary,Frequency,ReferenceDate,LastTransactionDate,Recency(Day),QuantitySum,QuantityPerTransaction,Sales/Transaction
0,12346,77183.60,1,2011-12-09 12:50:00,2011-01-18 10:01:00,325,74215,74215.000000,77183.600000
1,12747,4196.01,11,2011-12-09 12:50:00,2011-12-07 14:34:00,2,1275,115.909091,381.455455
2,12748,32317.32,206,2011-12-09 12:50:00,2011-12-09 12:20:00,0,25511,123.839806,156.880194
